In [4]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()

In [5]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [6]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

Descarga de empresas en sistema


In [7]:
llamadaEmpresaAntiguas = "SELECT Id,Empresa__r.Id, Empresa__r.Name, Empresa__r.Holding__c, Empresa__r.Campa_a__c, Empresa__r.Id_Empresa__c, Empresa__r.Empleador__c, Empresa__r.poliza__c, Empresa__r.RUT_Empresa__c, Empresa__r.Corredor__c,Empresa__r.Padre_Empresa__c, Name FROM Programas_por_Empresas__c WHERE Empresa__r.Padre_Empresa__c = false"
vllamadaEmpresa = sf.smart_query(llamadaEmpresaAntiguas)
vllamadaEmpresa.to_excel("F:/Empresas SF/reportes/empresas_Antiguas.xlsx", index=False)
vllamadaEmpresa.head()

  0%|          | 0/1 [00:00<?, ?it/s]

vllamadaEmpresa.shape


In [ ]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  SELECT Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  FROM account where ispersonaccount = false  AND Nueva_Estructura__c = False
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape

In [ ]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

In [ ]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

In [ ]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

In [ ]:
query2 = "select P_liza__c, Nombre_de_Empresa__c,Identificador__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c "

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]
df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()

fin empresas y comienzo de rescate de pacientes


In [ ]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone,PersonContactId  from Account where ispersonaccount = True 
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
        "PersonContactId",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
        "PersonContactId": "Id Contacto",
    }
)
Pacientes_SF.shape

In [ ]:
def siPrimerCero(y):
    if len(y) <= 4:
        return "NONE"
    if y[0] == "0":
        y = y[1:]
    return y

In [ ]:
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("–", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(".", "")
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].apply(lambda y: siPrimerCero(y))
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()

In [ ]:
Pacientes_Nones1 = Pacientes_SF[(Pacientes_SF["Rut"] == "NONE")]
Pacientes_SF = Pacientes_SF[(Pacientes_SF["Rut"] != "NONE")]
Pacientes_Nones1.to_excel("F:/Empresas SF/reportes/NONE1.xlsx", index=False)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


# buscar empresas desde Paciente_Programas\_\_c relaciones de empresas con pacientes

results = sf.query_all(
"""
select Id,Paciente**c, Programas_por_Empresas**c,padre**c from Paciente_Programas**c  
 """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
columns={
"Id": "Id Pacientes_Programas**c",
"Paciente**c": "Id Account",
"Programas_por_Empresas**c": "Programas_por_Empresas**c",
"padre\_\_c": "Nombre Campaña",
}
)
Paciente_Programas_SF.shape


# Union de Pacientes Programas y Programas por empresa

empresas_unidas = pd.merge(
left=Paciente_Programas_SF,
right=Programas_empresas,
how="left",
left_on="Programas_por_Empresas**c",
right_on="Programas_por_Empresas**c",
)
empresas_unidas.shape


Carga de Archivo carga


In [28]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "MetLife"
Pais = "Chile"
baseFinal = True
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo.head(5)

,poliza,empleador,rutE,RUTASEGURADO,DV,rut1,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut2,nacimiento,...,direccion,comuna,ciudad,TELEFONO,RELACION,NOMBRE,nombres,apellidos,email,celular
0,340007997,340007997,340007997,3391629,9,3391629-9,0,0,0-0,1937-07-11 00:00:00,...,EDUARDO ALERT 6229,LA REINA,SANTIAGO,965770703,CO,Beltrán Martínez Ana Eliana,Beltrán Martínez,Ana Eliana,luarayabeltran@gmail.com,965770703
1,340002495,340002495,340002495,6554272,2,6554272-2,0,0,0-0,1943-07-28 00:00:00,...,JUAN DE DIOS MALEBRAN 3744 CASA 51 CONDOMINIO...,PUENTE ALTO,NaN,0,CO,Vergara Mariqueo Sara,Vergara Mariqueo,Sara,patriciopineda46@gmail.com,982989836
2,340007997,340007997,340007997,5393043,3,5393043-3,0,0,0-0,1949-03-20 00:00:00,...,Los Fresnos 1355 Casa 2 Condominio La Llanura,HUECHURABA,SANTIAGO,56-2-22488062,CO,Muñoz Garate Edith Margot,Muñoz Garate,Edith Margot,AMADORITURRIETA@GMAIL.COM,56992279688
3,340007997,340007997,340007997,4888346,K,4888346-K,0,0,0-0,1950-08-13 00:00:00,...,BOCACCIO 544,LAS CONDES,SANTIAGO,2204262- 098740509,CO,Cabrera Undurraga Maria,Cabrera Undurraga,Maria,NaN,NaN
4,340007997,340007997,340007997,5473602,9,5473602-9,0,0,0-0,1952-05-24 00:00:00,...,AVENIDA FRANCISCO DE VILLAGRA 5824,LA REINA,SANTIAGO,NaN,CO,Contreras Mendoza Patricia,Contreras Mendoza,Patricia,NaN,NaN


In [29]:
Pacientes_Archivo.shape

(867670, 21)

In [30]:
Pacientes_IDrut = Pacientes_SF[
    [
        "Rut",
        "Id Account Paciente",
        "Id Contacto",
    ]
]

In [31]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("–", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(" ", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()

Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].astype(str)
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace("-", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace("–", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].astype(str)
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace(".", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace(" ", "")
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.upper()
Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.strip()
Pacientes_Archivo.shape

Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].astype(str)
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.upper()
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace("-", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace("–", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.replace(".", "")
Pacientes_Archivo["rutE"] = Pacientes_Archivo["rutE"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11032\1490231206.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11032\1490231206.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["poliza"] = Pacientes_Archivo["poliza"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11032\1490231206.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["ru

(867670, 22)

In [32]:
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rut2"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("-", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace("–", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].astype(str)
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(" ", "")
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.upper()
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.strip()
Pacientes_Archivo.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11032\2296516763.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].str.replace(".", "")


(867670, 23)

In [33]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("┐", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("═", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("╔", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("┌", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("┐", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("═", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("╔", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("┌", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11032\3410721566.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11032\3410721566.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [34]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo["rutT"] = Pacientes_Archivo["rutT"].apply(lambda y: siPrimerCero(y))
Pacientes_Archivo

# Pacientes_Nones = Pacientes_Archivo[(Pacientes_Archivo["rut"] == "NONE") | (Pacientes_Archivo["rutT"] == "NONE")]
# Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE") | (Pacientes_Archivo["rutT"] != "NONE")]

,poliza,empleador,rutE,RUTASEGURADO,DV,rut1,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut2,nacimiento,...,ciudad,TELEFONO,RELACION,NOMBRE,nombres,apellidos,email,celular,rut,rutT
0,340007997,340007997,340007997,3391629,9,3391629-9,0,0,0-0,1937-07-11 00:00:00,...,SANTIAGO,965770703,CO,Beltrán Martínez Ana Eliana,Beltrán Martínez,Ana Eliana,luarayabeltran@gmail.com,965770703,33916299,NONE
1,340002495,340002495,340002495,6554272,2,6554272-2,0,0,0-0,1943-07-28 00:00:00,...,NaN,0,CO,Vergara Mariqueo Sara,Vergara Mariqueo,Sara,patriciopineda46@gmail.com,982989836,65542722,NONE
2,340007997,340007997,340007997,5393043,3,5393043-3,0,0,0-0,1949-03-20 00:00:00,...,SANTIAGO,56-2-22488062,CO,Muñoz Garate Edith Margot,Muñoz Garate,Edith Margot,AMADORITURRIETA@GMAIL.COM,56992279688,53930433,NONE
3,340007997,340007997,340007997,4888346,K,4888346-K,0,0,0-0,1950-08-13 00:00:00,...,SANTIAGO,2204262- 098740509,CO,Cabrera Undurraga Maria,Cabrera Undurraga,Maria,NaN,NaN,4888346K,NONE
4,340007997,340007997,340007997,5473602,9,5473602-9,0,0,0-0,1952-05-24 00:00:00,...,SANTIAGO,NaN,CO,Contreras Mendoza Patricia,Contreras Mendoza,Patricia,NaN,NaN,54736029,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
867665,340023174,340023174,340023174,9999917,9,9999917-9,9999917,9,9999917-9,1965-11-28 00:00:00,...,NaN,NaN,AS,Urbina Lozano Alejandro,Urbina Lozano,Alejandro,aaurbinalozano@gmail.com,NaN,99999179,99999179
867666,340007852,340007852,340007852,9999921,7,9999921-7,9999921,7,9999921-7,1966-05-31 00:00:00,...,NaN,9-9699077,AS,Saez Espinoza Osvaldo Domingo,Saez Espinoza,Osvaldo Domingo,osaeze@bancochile.cl,999699077,99999217,99999217
867667,340007888,340007888,340007888,9999921,7,9999921-7,9999921,7,9999921-7,1966-05-31 00:00:00,...,NaN,9-9699077,AS,Saez Espinoza Osvaldo Domingo,Saez Espinoza,Osvaldo Domingo,osaeze@bancochile.cl,999699077,99999217,99999217
867668,340007856,340007856,340007856,9999921,7,9999921-7,9999921,7,9999921-7,1966-05-31 00:00:00,...,NaN,9-9699077,AS,Saez Espinoza Osvaldo Domingo,Saez Espinoza,Osvaldo Domingo,osaeze@bancochile.cl,999699077,99999217,99999217


In [ ]:
# Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo22.xlsx", index=False)

In [35]:
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rut"] != "NONE")]
Pacientes_Archivo = Pacientes_Archivo[(Pacientes_Archivo["rutT"] != "NONE")]
# | (Pacientes_Archivo["rutT"] != "NONE")


In [ ]:
Pacientes_Archivo["Llave"] = Pacientes_Archivo["rutT"] + Pacientes_Archivo["rut"]
Pacientes_Archivo["PolizaLlave"] = Pacientes_Archivo["poliza"] +  Pacientes_Archivo["rutT"] + Pacientes_Archivo["rut"]

Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/eliminar2.xlsx", index=False)

PAcientes Programas


In [36]:
query_pacientePrograma = (
    "SELECT Paciente__c,Programas_por_Empresas__r.Empresa__c,Paciente__r.IdentificationId__pc,Id from Paciente_Programas__c where padre__c =  '"
    + campaña
    + "'"
)
pacientePrograma = sf.smart_query(query_pacientePrograma)
pacientePrograma.head()

100%|██████████| 1/1 [02:38<00:00, 158.30s/it]


,Paciente__c,Programas_por_Empresas__r.Empresa__c,Paciente__r.IdentificationId__pc,Id
0,0018c00002amM6wAAE,0018c00002axQXrAAM,87177467,a1Y8c00000C5avSEAR
1,0018c00002amPGEAA2,0018c00002axQXrAAM,253741139,a1Y8c00000C5bQ1EAJ
2,0018c00002amPXdAAM,0018c00002axQXrAAM,141396692,a1Y8c00000C5bRmEAJ
3,0018c00002amQ3rAAE,0018c00002axQXrAAM,203297092,a1Y8c00000C5bUOEAZ
4,0018c00002amQICAA2,0018c00002axQXrAAM,209736918,a1Y8c00000C5bVTEAZ


In [37]:
pacientePrograma = pacientePrograma.rename(
    columns={
        "Id": "Id Paciente Programas",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Programas_por_Empresas__r.Empresa__c": "Id Account Empleador",
        "Paciente__c": "Id Account Paciente",
    }
)
pacientePrograma.head()

,Id Account Paciente,Id Account Empleador,Rut,Id Paciente Programas
0,0018c00002amM6wAAE,0018c00002axQXrAAM,87177467,a1Y8c00000C5avSEAR
1,0018c00002amPGEAA2,0018c00002axQXrAAM,253741139,a1Y8c00000C5bQ1EAJ
2,0018c00002amPXdAAM,0018c00002axQXrAAM,141396692,a1Y8c00000C5bRmEAJ
3,0018c00002amQ3rAAE,0018c00002axQXrAAM,203297092,a1Y8c00000C5bUOEAZ
4,0018c00002amQICAA2,0018c00002axQXrAAM,209736918,a1Y8c00000C5bVTEAZ


In [38]:
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("-", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace("–", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].astype(str)
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace(".", "")
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.upper()
pacientePrograma["Rut"] = pacientePrograma["Rut"].str.strip()
pacientePrograma.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11032\3172767859.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pacientePrograma["Rut"] = pacientePrograma["Rut"].str.replace(".", "")


(1011165, 4)

In [39]:
PacientesProgramas_PacientesSF = pd.merge(
    left=pacientePrograma,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)

PacientesProgramas_PacientesSF.head()

,Id Account Paciente,Id Account Empleador,Rut_x,Id Paciente Programas,Rut_y,Id Contacto
0,0018c00002amM6wAAE,0018c00002axQXrAAM,87177467,a1Y8c00000C5avSEAR,87177467,0038c000035CVmaAAG
1,0018c00002amPGEAA2,0018c00002axQXrAAM,253741139,a1Y8c00000C5bQ1EAJ,253741139,0038c000035CYwXAAW
2,0018c00002amPXdAAM,0018c00002axQXrAAM,141396692,a1Y8c00000C5bRmEAJ,141396692,0038c000035CZDpAAO
3,0018c00002amQ3rAAE,0018c00002axQXrAAM,203297092,a1Y8c00000C5bUOEAZ,203297092,0038c000035CZk5AAG
4,0018c00002amQICAA2,0018c00002axQXrAAM,209736918,a1Y8c00000C5bVTEAZ,209736918,0038c000035CZyIAAW


account contact relation


In [40]:
query_accountContact = (
    "select Id,ContactId, AccountId, RutContact__c ,Contact.AccountId from AccountContactRelation where Account.Campa_a__c = '"
    + campaña
    + "'"
)
accountContactRelation = sf.smart_query(query_accountContact)
accountContactRelation.head()

100%|██████████| 1/1 [02:08<00:00, 128.87s/it]


,Id,ContactId,AccountId,RutContact__c,Contact.AccountId
0,07k8c00000RGnshAAD,0038c000035CYwXAAW,0018c00002axQXrAAM,253741139,0018c00002amPGEAA2
1,07k8c00000RGnskAAD,0038c000035CVmaAAG,0018c00002axQXrAAM,87177467,0018c00002amM6wAAE
2,07k8c00000RGnslAAD,0038c000035CbnFAAS,0018c00002axQXrAAM,122766802,0018c00002amS79AAE
3,07k8c00000RGnsmAAD,0038c000035CZDpAAO,0018c00002axQXrAAM,141396692,0018c00002amPXdAAM
4,07k8c00000RGnsnAAD,0038c000035CZyIAAW,0018c00002axQXrAAM,209736918,0018c00002amQICAA2


In [41]:
accountContactRelation = accountContactRelation.rename(
    columns={
        "Id": "Id AccountContactRelation",
        "ContactId": "Id Contact Paciente",
        "AccountId": "Id Account Empresa",
        "RutContact__c": "Rut",
        "Contact.AccountId": "Id Account Paciente",
    }
)
accountContactRelation.head()

,Id AccountContactRelation,Id Contact Paciente,Id Account Empresa,Rut,Id Account Paciente
0,07k8c00000RGnshAAD,0038c000035CYwXAAW,0018c00002axQXrAAM,253741139,0018c00002amPGEAA2
1,07k8c00000RGnskAAD,0038c000035CVmaAAG,0018c00002axQXrAAM,87177467,0018c00002amM6wAAE
2,07k8c00000RGnslAAD,0038c000035CbnFAAS,0018c00002axQXrAAM,122766802,0018c00002amS79AAE
3,07k8c00000RGnsmAAD,0038c000035CZDpAAO,0018c00002axQXrAAM,141396692,0018c00002amPXdAAM
4,07k8c00000RGnsnAAD,0038c000035CZyIAAW,0018c00002axQXrAAM,209736918,0018c00002amQICAA2


In [42]:
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("-", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace("–", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].astype(str)
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace(".", "")
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.upper()
accountContactRelation["Rut"] = accountContactRelation["Rut"].str.strip()
accountContactRelation.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11032\2268980097.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  accountContactRelation["Rut"] = accountContactRelation["Rut"].str.replace(".", "")


(916786, 5)

In [43]:
AccountContact_PacientesSF = pd.merge(
    left=accountContactRelation,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
AccountContact_PacientesSF.head()

,Id AccountContactRelation,Id Contact Paciente,Id Account Empresa,Rut_x,Id Account Paciente,Rut_y,Id Contacto
0,07k8c00000RGnshAAD,0038c000035CYwXAAW,0018c00002axQXrAAM,253741139,0018c00002amPGEAA2,253741139,0038c000035CYwXAAW
1,07k8c00000RGnskAAD,0038c000035CVmaAAG,0018c00002axQXrAAM,87177467,0018c00002amM6wAAE,87177467,0038c000035CVmaAAG
2,07k8c00000RGnslAAD,0038c000035CbnFAAS,0018c00002axQXrAAM,122766802,0018c00002amS79AAE,122766802,0038c000035CbnFAAS
3,07k8c00000RGnsmAAD,0038c000035CZDpAAO,0018c00002axQXrAAM,141396692,0018c00002amPXdAAM,141396692,0038c000035CZDpAAO
4,07k8c00000RGnsnAAD,0038c000035CZyIAAW,0018c00002axQXrAAM,209736918,0018c00002amQICAA2,209736918,0038c000035CZyIAAW


empresa contacto convenio


In [44]:
query_empresacontactoconvenio = (
    "select Id,Paciente__r.IdentificationId__pc, Empresa__r.P_liza__c,Empresa__r.Rut_Empresa__c, Empresa__c, Paciente__c,N_Documento_Titular__c from EmpresaContactoConvenios__c where Empresa__r.Campa_a__c = '"
    + campaña
    + "'"
)
empresacontactoconvenio = sf.smart_query(query_empresacontactoconvenio)
empresacontactoconvenio.head()

100%|██████████| 1/1 [02:30<00:00, 150.39s/it]


,Id,Paciente__r.IdentificationId__pc,Empresa__r.P_liza__c,Empresa__r.Rut_Empresa__c,Empresa__c,Paciente__c,N_Documento_Titular__c
0,a40RN000000q575YAA,199175742,340023542,-,a3yRN0000006sFLYAY,0018c00002bJkewAAC,199175742
1,a40RN000000q8BDYAY,15638903K,340023542,-,a3yRN0000006sFLYAY,0018c00002b043oAAA,15638903K
2,a40RN000000qC04YAE,184271516,340023542,-,a3yRN0000006sFLYAY,0018c00002b049FAAQ,176591048
3,a40RN000000qFbOYAU,272172277,340023542,-,a3yRN0000006sFLYAY,0018c00002oNqQxAAK,272172277
4,a40RN000000qGwJYAU,197504749,340023542,-,a3yRN0000006sFLYAY,0018c00002cXFvaAAG,197504749


In [45]:
empresacontactoconvenio = empresacontactoconvenio.rename(
    columns={
        "Id": "Id EmpresaContactoConvenio",
        "Paciente__r.IdentificationId__pc": "Rut",
        "Empresa__c": "Id Convenio Empresa",
        "Paciente__c": "Id Account Paciente",
        "N_Documento_Titular__c": "Rut Titular",
        "Empresa__r.P_liza__c":"Poliza",
        "Empresa__r.Rut_Empresa__c":"Rut Empresa",
    }
)
empresacontactoconvenio.head()

,Id EmpresaContactoConvenio,Rut,Poliza,Rut Empresa,Id Convenio Empresa,Id Account Paciente,Rut Titular
0,a40RN000000q575YAA,199175742,340023542,-,a3yRN0000006sFLYAY,0018c00002bJkewAAC,199175742
1,a40RN000000q8BDYAY,15638903K,340023542,-,a3yRN0000006sFLYAY,0018c00002b043oAAA,15638903K
2,a40RN000000qC04YAE,184271516,340023542,-,a3yRN0000006sFLYAY,0018c00002b049FAAQ,176591048
3,a40RN000000qFbOYAU,272172277,340023542,-,a3yRN0000006sFLYAY,0018c00002oNqQxAAK,272172277
4,a40RN000000qGwJYAU,197504749,340023542,-,a3yRN0000006sFLYAY,0018c00002cXFvaAAG,197504749


In [46]:
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("-", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace("–", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].astype(str)
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(" ", "")
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.upper()
empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.strip()

empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].astype(str)
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace("-", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace("–", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].astype(str)
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace(".", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace(" ", "")
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.upper()
empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.strip()

empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].astype(str)
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace("-", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace("–", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].astype(str)
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace(".", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.replace(" ", "")
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.upper()
empresacontactoconvenio["Poliza"] = empresacontactoconvenio["Poliza"].str.strip()


empresacontactoconvenio.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11032\3330564967.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Rut"] = empresacontactoconvenio["Rut"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11032\3330564967.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  empresacontactoconvenio["Rut Empresa"] = empresacontactoconvenio["Rut Empresa"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_11032\3330564967.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when r

(833462, 7)

In [47]:
empresacontactoconvenio_PacientesSF = pd.merge(
    left=empresacontactoconvenio,
    right=Pacientes_IDrut,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account Paciente",
)
empresacontactoconvenio_PacientesSF.head()

,Id EmpresaContactoConvenio,Rut_x,Poliza,Rut Empresa,Id Convenio Empresa,Id Account Paciente,Rut Titular,Rut_y,Id Contacto
0,a40RN000000q575YAA,199175742,340023542,,a3yRN0000006sFLYAY,0018c00002bJkewAAC,199175742,199175742,0038c000035v8MPAAY
1,a40RN000000q8BDYAY,15638903K,340023542,,a3yRN0000006sFLYAY,0018c00002b043oAAA,15638903K,15638903K,0038c000035sDzdAAE
2,a40RN000000qC04YAE,184271516,340023542,,a3yRN0000006sFLYAY,0018c00002b049FAAQ,176591048,184271516,0038c000035sE4qAAE
3,a40RN000000qFbOYAU,272172277,340023542,,a3yRN0000006sFLYAY,0018c00002oNqQxAAK,272172277,272172277,0038c00003KEDi8AAH
4,a40RN000000qGwJYAU,197504749,340023542,,a3yRN0000006sFLYAY,0018c00002cXFvaAAG,197504749,197504749,0038c000036lOhDAAU


In [ ]:
""" if(campaña!="MetLife"):
    pacientePrograma.to_excel("F:/Empresas SF/reportes/paciente_programas.xlsx", index=False) """

In [ ]:
empresacontactoconvenio_PacientesSF.shape

Rescatamos todos los ruts Id con su empresa desde la campaña


In [48]:
Pacientes_Archivo.head()

,poliza,empleador,rutE,RUTASEGURADO,DV,rut1,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut2,nacimiento,...,ciudad,TELEFONO,RELACION,NOMBRE,nombres,apellidos,email,celular,rut,rutT
148,340004490,340004490,340004490,14090568,2,14090568-2,100000008,2,100000008-2,2015-02-01 00:00:00,...,NaN,994297864,HI,Lopez Herrera Catalina,Lopez Herrera,Catalina,arielcaro25@gmail.com,994297864,140905682,1000000082
149,330000004,330000004,330000004,10988450,2,10988450-2,1000000,9,1000000-9,2008-06-20 00:00:00,...,santiago,2-225754,HI,Asignacion Maternal Asignacion Mat,Asignacion Maternal,Asignacion Mat,sdiaz@achs.cl,974764119,109884502,10000009
150,340016437,340016437,340016437,10527924,8,10527924-8,10000020,2,10000020-2,1972-04-07 00:00:00,...,NaN,NaN,CO,Muñoz Ulloa Miriam Del Pilar,Muñoz Ulloa,Miriam Del Pilar,rfariasacuna@gmail.com,NaN,105279248,100000202
151,340023268,340023268,340023268,10000035,0,10000035-0,10000035,0,10000035-0,1965-04-03 00:00:00,...,NaN,NaN,AS,Hernandez Vera Mauricio,Hernandez Vera,Mauricio,NaN,NaN,100000350,100000350
152,340020470,340020470,340020470,10000040,7,10000040-7,10000040,7,10000040-7,1980-05-09 00:00:00,...,NaN,998023469,AS,Valenzuela Luco Catherine Andrea,Valenzuela Luco,Catherine Andrea,valenzuela.catherine@gmail.com,998023469,100000407,100000407


Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)


Pacientes_Archivo.drop_duplicates(
[
"rut",
"rutT",
"LLaveEmpresa",
"LLAveRut",
],
keep="last",
)
Pacientes_Archivo.shape


In [49]:
print(campaña)

MetLife


In [50]:
Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

In [ ]:
# Pacientes_Archivo.to_excel("F:/Empresas SF/reportes/Pacientes_Archivo.xlsx", index=False)

In [51]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
# Pacientes_con_id.to_excel("F:/Empresas SF/reportes/Pacientes_con_id.xlsx", index=False)
Pacientes_con_id.shape

(865458, 34)

In [52]:
Pacientes_con_id.head()

,poliza,empleador,rutE,RUTASEGURADO,DV,rut1,RUTBENEFICIARIO,RUTBENEFIARIO_DV,rut2,nacimiento,...,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono,Id Contacto
0,340004490,340004490,340004490,14090568,2,14090568-2,100000008,2,100000008-2,2015-02-01 00:00:00,...,001RN000006l2JUYAY,140905682,Lopez Herrera Agustina,RUT,None,None,None,Mujer,994297864,003RN00000JgA2CYAV
1,330000004,330000004,330000004,10988450,2,10988450-2,1000000,9,1000000-9,2008-06-20 00:00:00,...,0018c00002axzqHAAQ,109884502,Soraya Milka Diaz Cabrera,RUT,None,None,None,Mujer,None,0038c000035q9QSAAY
2,340016437,340016437,340016437,10527924,8,10527924-8,10000020,2,10000020-2,1972-04-07 00:00:00,...,0018c00002bHbknAAC,105279248,Farias Acuña Ricardo,RUT,None,None,None,Hombre,None,0038c000035sz9SAAQ
3,340023268,340023268,340023268,10000035,0,10000035-0,10000035,0,10000035-0,1965-04-03 00:00:00,...,0018c00002eTktlAAC,100000350,Mauricio Eduardo Hernandez Vera,RUT,None,None,None,Hombre,None,0038c000038NlMJAA0
4,340020470,340020470,340020470,10000040,7,10000040-7,10000040,7,10000040-7,1980-05-09 00:00:00,...,0018c00002mVb0vAAC,100000407,Luco Catherine Andrea Valenzuela,RUT,None,None,None,Mujer,998023469,0038c00003IgE6PAAV


In [ ]:
# Pacientes_con_id.to_excel("F:/Empresas SF/reportes/id.xlsx", index=False)

In [53]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")


def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""


def fechaValida(fecha):
    if (len(str(fecha)) <= 9 ):
        fecha = ""
        
    fecha = str(fecha)
    fecha1 = str(fecha)
    
    #datetime.strptime(fecha, '%Y-%m-%d')
    
    try:
        fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        date_string = fecha1
        print("date_string =", date_string)
        print("type of date_string =", type(date_string))

        date_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")

        print("date_object =", date_object)
        print("type of date_object =", type(date_object))
        return date_object
    except ValueError:
        return datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    

In [3]:
noestan = pd.DataFrame()
Pacientes_con_id["dertyuiop"] = Pacientes_con_id["nacimiento"]
Pacientes_con_id["derty"] = Pacientes_con_id["nacimiento"].apply(lambda y: fechaValida(y))
Pacientes_con_id.head()
noestan = Pacientes_con_id[(Pacientes_con_id["RutEstaoNo"] == False)]

NameError: name 'Pacientes_con_id' is not defined

In [54]:


# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
# Subir_pacientes["Email_Seteado__c"] = ""
#Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]

# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in noestan.columns) == True:
    Subir_pacientes["FirstName"] = noestan["nombres"]
if ("apellidos" in noestan.columns) == True:
    Subir_pacientes["LastName"] = noestan["apellidos"]
if ("rut" in noestan.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        noestan["rut"].astype(str).str.upper()
    )


if ("email" in noestan.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = noestan["email"].astype(str).str.lower()
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(
        lambda y: isValid(y)
    )
    Subir_pacientes["Email_Seteado__c"] = np.where(
        Subir_pacientes["PersonEmail"] != "", "True", "False"
    )

    # Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
#if ("nacimiento" in Pacientes_con_id.columns) == True:    
#    Subir_pacientes["PersonBirthdate"] = pd.to_datetime(Pacientes_con_id["nacimiento"].astype(str), format="%Y-%m-%d")  
if ("nacimiento" in noestan.columns) == True:    
    #try:
    Subir_pacientes["PersonBirthdate"] = noestan["nacimiento"].apply(lambda y: fechaValida(y))
    Subir_pacientes["derty"] = noestan["nacimiento"].apply(lambda y: fechaValida(y))
    #except ValueError:
        #Subir_pacientes["PersonBirthdate"] = pd.to_datetime("1900-01-01", format="%Y-%m-%d")
 
if ("sexo" in noestan.columns) == True:
    Sexo = {
        "H": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "Prefiero no responder",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "M": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon": "Hombre",
    }
    noestan.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = noestan["sexo"]
if ("direccion" in noestan.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        noestan["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in noestan.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        noestan["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in noestan.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        noestan["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = noestan["cargo"].str.title()

if ("division" in noestan.columns) == True:
    Subir_pacientes["PersonDepartment"] = noestan["division"].str.title()

if ("celular" in noestan.columns) == True:
    Subir_pacientes["Phone"] = (
        noestan["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace(".", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("null", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NULL", "")


Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"

#Subir_pacientes_Que_no_Estan = Subir_pacientes
#Subir_pacientes_Estan = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == True]
Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]

#Subir_pacientes["PersonBirthdate"] = Subir_pacientes["PersonBirthdate"].str.replace("0001-01-01 00:00:00", "")

# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes = Subir_pacientes.drop_duplicates(
    [
        "IdentificationId__pc",
    ],
    keep="last",
)
Subir_pacientes.to_excel(
    "F:/Empresas SF/reportes/Pacientes_Que_No_Estaban.xlsx", index=False
)
Subir_pacientes.head()

NameError: name 'noestan' is not defined

In [ ]:
Subir_pacientes.shape

In [ ]:
def traerId(rut):
    if len(rut) >= 8:
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc where IdentificationId__pc = '" + rut + "')"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return "None"


def traerContact(ide):
    ide = str(ide)
    if len(ide) >= 12:
        idcontact = sf.Account.get(ide)
        return idcontact["PersonContactId"]
    else:
        return "NONES"


def agregarPaciente(listado):
    vListado = listado.split(",")
    
    derty = None
    #vListado[9] = datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S")
        
        
    if (vListado[9] == ""):                               
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],  
                                          
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    else:
        try:
            derty = dict(
                sf.Account.create(
                    # derty = sf.Account.create(
                    {
                        "FirstName": vListado[0],
                        "LastName": vListado[1],
                        "IdentificationId__pc": vListado[2],
                        "RecordTypeId": vListado[3],
                        "BillingCountry": vListado[4],
                        "Identification_Type__c": vListado[5],
                        "Pa_s_del_Paciente__c": vListado[6],
                        "Cuenta_validada__c": True,
                        "PersonEmail": vListado[8],
                        "PersonBirthdate": vListado[9] ,
                        #"PersonBirthdate":datetime.strptime(vListado[9], "%Y-%m-%d %H:%M:%S").formatGMT('yyyy-MM-dd\'T\'HH:mm:ss.SSS\'Z\'' + '"'),
                        "HealthCloudGA__Gender__pc": vListado[10],
                        "Phone": vListado[11],
                        "BillingCity": vListado[12],
                        "BillingState": vListado[13],
                        "BillingStreet": vListado[14],
                        "Email_Seteado__c": bool(vListado[15]),
                    }
                )
            )
            # return derty["id"]
        except SalesforceMalformedRequest as e:
            print(f"Malformed request error: {e}")
            error = {e}

        if derty is not None:
            return derty["id"]
        else:
            traerId(vListado[2])
    # Verify that the Contact was created successfully
    # created_contact = sf.Contact.get(sf_res['id'])
    # print(created_contact)


""" sf_res =None
try:
    # Create the new Contact record in Salesforce
    sf_res = sf.Contact.create(new_contact)
except SalesforceMalformedRequest as e:
    if 'duplicateResut' in e[0]:
        print(f"Malformed request error: {e}")
        # Handle the error here as needed
        #Bypass SF Email duplicate rule by removing the email to create a SF Lead
        del new_contact['Email']
        sf_res = sf.Contact.create(new_contact)


if sf_res is not None:
    # Verify that the Contact was created successfully
    created_contact = sf.Contact.get(sf_res['id'])
    print(created_contact) """

In [ ]:
Subir_pacientes.head()

In [ ]:
Subir_pacientes = Subir_pacientes.drop_duplicates(
    [
        "IdentificationId__pc",
    ],
    keep="last",
)

Subir_pacientes["Id"] = Subir_pacientes["IdentificationId__pc"].apply(
    lambda y: traerId(y)
)

Subir_pacientes.shape

In [ ]:
Subir_pacientes["Id"] = Subir_pacientes["Id"].astype(str)
Subir_pacientes = Subir_pacientes[Subir_pacientes["Id"].str.len() <= 12]
# Subir_pacientes["numero"] = Subir_pacientes["Id"].str.len()
Subir_pacientes.shape

In [ ]:
#Subir_pacientes["dato_account"].head()

In [ ]:
Subir_pacientes.to_excel("F:/Empresas SF/reportes/Todesffff.xlsx", index=False)

In [ ]:
Subir_pacientes["dato_account"] = ""
if ("FirstName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + Subir_pacientes[
        "FirstName"
    ].astype(str)
if ("LastName" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["LastName"].astype(str)
    )
if ("IdentificationId__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["IdentificationId__pc"].astype(str)
    )
if ("RecordTypeId" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["RecordTypeId"].astype(str)
    )
if ("BillingCountry" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCountry"].astype(str)
    )
if ("Identification_Type__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Identification_Type__c"].astype(str)
    )
if ("Pa_s_del_Paciente__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Pa_s_del_Paciente__c"].astype(str)
    )
if ("Cuenta_validada__c" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Cuenta_validada__c"].astype(str)
    )

if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["PersonEmail"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

if ("PersonBirthdate" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (Subir_pacientes["dato_account"] + "," + Subir_pacientes["PersonBirthdate"].apply(lambda x: x.isoformat() + "Z"))
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("HealthCloudGA__Gender__pc" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["HealthCloudGA__Gender__pc"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("Phone" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"] + "," + Subir_pacientes["Phone"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingCity" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingCity"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingState" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingState"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("BillingStreet" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["BillingStreet"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","
if ("PersonEmail" in Subir_pacientes.columns) == True:
    Subir_pacientes["dato_account"] = (
        Subir_pacientes["dato_account"]
        + ","
        + Subir_pacientes["Email_Seteado__c"].astype(str)
    )
else:
    Subir_pacientes["dato_account"] = Subir_pacientes["dato_account"] + ","

# Subir_pacientes["dato_account"] =  Subir_pacientes["dato_account"] =

Subir_pacientes["Id_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
)
""" Subir_pacientes["Contact_creado"] = Subir_pacientes["dato_account"].apply(
    lambda x: agregarPaciente(x)
) """

In [ ]:
query2 = "select P_liza__c, Nombre_de_Empresa__c,Identificador__c, Rut_Empresa__c, Empresa_Padre__r.Name, Empresa__r.name,Holding__c, Plan__c, Pais__c, Activo__c, Id from Datos_Empresa__c WHERE Empresa_Padre__r.Name = '" + campaña + "'"

df2 = sf.smart_query(query2)
df2["LLave"] = df2["Rut_Empresa__c"].str.upper().str.replace("-", "") + df2["P_liza__c"]

df2["dato"] = 1
df2.to_excel("F:/Empresas SF/reportes/empresas_asociadas.xlsx", index=False)
df2.head()

In [ ]:
df2.shape

In [ ]:
Subir_empresacontactoconvenio.head()

In [ ]:
Pacientes_con_id_no = Pacientes_con_id[Pacientes_con_id["RutEstaoNo"] == False]
Pacientes_con_id_no["Id Account Paciente"] = Pacientes_con_id_no["rut"].apply(
    lambda y: traerId(y)
)
Pacientes_con_id_no["Id Contacto"] = Pacientes_con_id_no["Id Account Paciente"].apply(
    lambda y: traerContact(y)
)
Pacientes_con_id_no.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_no.xlsx", index=False
)

Pacientes_con_id_si = Pacientes_con_id[Pacientes_con_id["RutEstaoNo"] == True]
Pacientes_con_id_si.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_si.xlsx", index=False
)

Pacientes_con_id_final = pd.concat(
    [
        Pacientes_con_id_si,
        Pacientes_con_id_no,
    ],
    axis=0,
)

Pacientes_con_id_final.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_final.xlsx", index=False
)

if campaña == "Southbridge Plan Mujer":
    VpacientePrograma = "a1WRN0000007YwD2AU"
    VempresaaAccount = "001RN000003rwVaYAI"
    VempresaConvenio = "a3yRN0000006sVTYAY"
elif campaña == "Southbridge":
    VpacientePrograma = "a1W8c0000083pM9EAI"
    VempresaaAccount = "0018c00002f3Ev2AAE"
    VempresaConvenio = "a3yRN0000006sqYYAQ"
elif campaña == "Construmart":
    VpacientePrograma = "a1W8c0000083q9TEAQ"
    VempresaaAccount = "0018c00002ghJgBAAU"
    VempresaConvenio = "a3yRN0000006rlZYAQ"
elif campaña == "EY":
    VpacientePrograma = "a1W8c0000083laZEAQ"
    VempresaaAccount = "0018c00002eTboVAAS"
    VempresaConvenio = "a3yRN0000006rsTYAQ"
elif campaña == "Colmena":
    VpacientePrograma = "a1W8c0000083j8rEAA"
    VempresaaAccount = "0018c00002cXclQAAS"
    VempresaConvenio = "a3yRN0000006rf5YAA"
elif campaña == "Pedro De Valdivia":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006sVzYAI"
elif campaña == "Forum":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006s3SYAQ"
elif campaña == "Cruz Verde":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006rsqYAA"
elif campaña == "Milan Fajanovic":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006sR4YAI"
elif campaña == "ProChile":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006rpsYAA"
elif campaña == "Farmacias Ahumada":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006rspYAA"
elif campaña == "Minera El Abra":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006snOYAQ"
elif campaña == "Instruvalve":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006s9DYAQ"
elif campaña == "VidaCel":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006suzYAA"
elif campaña == "Care Assistance Colaboradores":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006reXYA"
elif campaña == "Nexo":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006sORYAY"
elif campaña == "Talo":
    VpacientePrograma = "a1W8c0000083iWiEAI"
    VempresaaAccount = "0018c00002cXJTsAAO"
    VempresaConvenio = "a3yRN0000006t10YAA"
else:
    VpacientePrograma = "Buscarv"
    VempresaaAccount = "Buscarv"
    VempresaConvenio = "Buscarv"

#Programas por empresas
Pacientes_con_id_final["esonoes"] = Pacientes_con_id_final["rut"].isin(
    PacientesProgramas_PacientesSF["Rut_y"]
)
""" Subir_programas_empresa = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes"] == False
]
Subir_programas_empresa["Paciente__c"] = Subir_programas_empresa["Id Account Paciente"]
Subir_programas_empresa["Programas_por_Empresas__c"] = VpacientePrograma
Subir_programas_empresa = Subir_programas_empresa[
    [
        # "empresa",
        "poliza",
        "Paciente__c",
        "Programas_por_Empresas__c",
    ]
]
Subir_programas_empresa.to_excel(
    "F:/Empresas SF/reportes/programas_empresa.xlsx", index=False
)
 """
#
Pacientes_con_id_final["esonoes1"] = Pacientes_con_id_final["rut"].isin(
    AccountContact_PacientesSF["Rut_y"]
)
Subir_AccountContact = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes1"] == False
]
Subir_AccountContact["ContactId"] = Subir_AccountContact["Id Contacto"]
Subir_AccountContact["AccountId"] = VempresaaAccount

Subir_AccountContact = Subir_AccountContact[
    [
        # "empresa",
        "poliza",
        "ContactId",
        "AccountId",
    ]
]
Subir_AccountContact.to_excel(
    "F:/Empresas SF/reportes/account_Contact.xlsx", index=False
)
if(campaña == "MetLife"):
    empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Poliza"] + empresacontactoconvenio_PacientesSF["Rut Empresa"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
    Pacientes_con_id_final["llaveA"] = Pacientes_con_id_final["poliza"] + Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"]
    Pacientes_con_id_final["llavepa"] = Pacientes_con_id_final["poliza"] + Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"]
    empresacontactoconvenio_PacientesSF["polizallavepa"] = empresacontactoconvenio_PacientesSF["Poliza"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
elif(campaña == "Vida Security"):
    empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Poliza"] +empresacontactoconvenio_PacientesSF["Rut Empresa"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
    Pacientes_con_id_final["llaveA"] = Pacientes_con_id_final["poliza"] + Pacientes_con_id_final["rutE"] + Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"] 
    empresacontactoconvenio_PacientesSF["polizallavepa"] = empresacontactoconvenio_PacientesSF["Poliza"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
    Pacientes_con_id_final["llavepa"] = Pacientes_con_id_final["poliza"] + Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"]

    
else:
    empresacontactoconvenio_PacientesSF["llave"] = empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
    Pacientes_con_id_final["llaveA"] = Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"]    
    empresacontactoconvenio_PacientesSF["polizallavepa"] = empresacontactoconvenio_PacientesSF["Poliza"] + empresacontactoconvenio_PacientesSF["Rut Titular"] + empresacontactoconvenio_PacientesSF["Rut_y"]
    Pacientes_con_id_final["llavepa"] = Pacientes_con_id_final["poliza"] + Pacientes_con_id_final["rutT"] + Pacientes_con_id_final["rut"]




Pacientes_con_id_final["esonoes2"] = empresacontactoconvenio_PacientesSF["llave"].isin(
    Pacientes_con_id_final["llaveA"]
)
Pacientes_con_id_final["sionose"] = Pacientes_con_id_final["llavepa"].isin(
    empresacontactoconvenio_PacientesSF["polizallavepa"]
)

Subir_empresacontactoconvenio = Pacientes_con_id_final[
    Pacientes_con_id_final["esonoes2"] == False
]
Subir_empresacontactoconvenio2 = Pacientes_con_id_final[
    Pacientes_con_id_final["sionose"] == False
]
#Subir_empresacontactoconvenio = Pacientes_con_id_final

Subir_empresacontactoconvenio["Paciente__c"] = Subir_empresacontactoconvenio[
    "Id Account Paciente"
]

Subir_empresacontactoconvenio["N_Documento_Titular__c"] = Subir_empresacontactoconvenio[
    "rutT"
]
Subir_empresacontactoconvenio["Empresa__c"] = VempresaConvenio

Subir_empresacontactoconvenio = Subir_empresacontactoconvenio[
    [
        "empleador",
        "rutE",
        "poliza",
        "Paciente__c",
        "N_Documento_Titular__c",
        "Empresa__c",
        "esonoes2",
        #"llave",
        
    ]
]
Subir_empresacontactoconvenio.to_excel(
    "F:/Empresas SF/reportes/empresacontactoconvenio.xlsx", index=False)
Pacientes_con_id_final.to_excel(
    "F:/Empresas SF/reportes/Pacientes_con_id_final2.xlsx", index=False
    
)
Subir_empresacontactoconvenio2.to_excel(
    "F:/Empresas SF/reportes/empresacontactoconvenio2.xlsx", index=False)

Cuadrar eliminar pacientes q no estan en el listado
//-----------------------------------------------------------//


In [ ]:
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "163820064")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "153298939")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "191710940")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "182470082")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "16277865")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "16277865K")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "HEGJ930928HMNRRL06")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "16007258K")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "86676540")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "202201466")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "105233469")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "130949185")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "255356798")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "34005945")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "185410072")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "197415029")].index, inplace=True)
empresacontactoconvenio_PacientesSF.drop(empresacontactoconvenio_PacientesSF[(empresacontactoconvenio_PacientesSF["Rut_y"] == "254468134")].index, inplace=True)



empresacontactoconvenio_PacientesSF.head()

In [ ]:
empresacontactoconvenio_PacientesSF.to_excel(
    "F:/Empresas SF/reportes/SFpacientes.xlsx", index=False)

In [ ]:
contactd = sf.Account.get("0018c00002lUyo0AAC")
contactd["PersonContactId"]

In [ ]:
# contactd["PersonContactId"]
print(datetime.now())

In [ ]:
Subir_pacientes.to_excel("F:/Empresas SF/reportes/tod  es.xlsx", index=False)

query2 = "select P_liza**c, Nombre_de_Empresa**c, Rut_Empresa**c, Empresa_Padre**r.Name, Empresa**r.name,Holding**c, Plan**c, Pais**c, Activo**c, Id from Datos_Empresa**c "

df2 = sf.smart_query(query2)
df2.to_excel("F:/Empresas SF/reportes/empresas_nuevas.xlsx", index=False)
df2.head()


Subir_pacientes.to_excel("F:/Empresas SF/reportes/111account.xlsx", index=False)


derty = Subir_pacientes["IdentificationId__pc"]
sf.Contact.create({"IdentificationId\_\_pc": "derty", "Email": "example@example.com"})


Subir_pacientes.head()


Subir_pacientes.head()
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account\_\_l.xlsx", index=False)


In [ ]:
""" if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )
 """

In [ ]:
""" if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    ) """

In [ ]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [ ]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)

In [ ]:
campaña_en_pacientes2["rut"].shape

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
rutes = pd.DataFrame(columns=["rut"])
rutes["rut"] = Pacientes_Archivo["rut"]
rutes["rutT"] = Pacientes_Archivo["rut2"]
rutes["Contratante"] = Pacientes_Archivo["Contratante"]
# rutes["poliza"] = Pacientes_Archivo["poliza"]
rutes.head()

In [ ]:
rutes["rut"] = rutes["rut"].astype(str)
rutes["rut"] = rutes["rut"].str.replace(".", "")
rutes["rut"] = rutes["rut"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rut"] = rutes["rut"].str.upper()
rutes["rut"] = rutes["rut"].str.strip()

rutes["rutT"] = rutes["rutT"].astype(str)
rutes["rutT"] = rutes["rutT"].str.replace(".", "")
rutes["rutT"] = rutes["rutT"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["rutT"] = rutes["rutT"].str.upper()
rutes["rutT"] = rutes["rutT"].str.strip()

""" rutes["poliza"] = rutes["poliza"].astype(str)
rutes["poliza"] = rutes["poliza"].str.replace(".", "")
rutes["poliza"] = rutes["poliza"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
rutes["poliza"] = rutes["poliza"].str.strip() """


rutes.head()

In [ ]:
# derty = rutes.to_numpy().transpose().tolist()


Pacientes_SF2 = Pacientes_SF[["Rut", "Id Account Paciente"]]

In [ ]:
rutesi = pd.merge(
    left=rutes,
    right=Pacientes_SF2,
    how="left",
    left_on="rut",
    right_on="Rut",
)
rutesi.shape
rutesi.to_excel("F:/Empresas SF/reportes/rutsyempresa.xlsx", index=False)

In [ ]:
rutesi.head()

In [ ]:
rutesi_antg = pd.merge(
    left=rutesi,
    right=empresas_unidas,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)

In [ ]:
rutesi_antg.head()

In [ ]:
# rutesi_antg .to_excel("F:/Empresas SF/reportes/ruts_empresas_antiguos.xlsx", index=False)

In [ ]:
import json
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
from json import loads, dumps
import simpler_sf

simpler_sf.simple_salesforce()

In [ ]:
with open("login.json", "r") as j:
    misDatos = json.load(j)
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [ ]:
""" query = "select Id, Account.Name, RutContact__c from AccountContactRelation "

df1 = sf.smart_query(query) """

In [ ]:
# nuevosrutes = "F:/Empresas SF/reportes/rut.xlsx"
# campaña = "BCI Seguros Colectivos"
# Pais = "Chile"
# Pais = "México"

# rutes = pd.read_excel(nuevosrutes)
# rutes = Subir_pacientes["rut"]

In [ ]:
""" df1.head() """

In [ ]:
""" df1["rut"] = df1["RutContact__c"].astype(str)
df1["rut"] = df1["rut"].str.replace(".", "")
df1["rut"] = df1["rut"].str.replace(
    "-", ""
)  # aca se trabaja con el nuevo rut desde rut1 ojooooooo
df1["rut"] = df1["rut"].str.upper()
df1["rut"] = df1["rut"].str.strip()
df1.head() """

In [ ]:
""" rutes.head() """

In [ ]:
""" rutesi = pd.merge(
    left=rutes,
    right=df1,
    how="left",
    left_on="rut",
    right_on="rut",
) """

In [ ]:
""" rutesi.to_excel("F:/Empresas SF/reportes/rutesnuevos_accountcontactrelation.xlsx", index=False) """

In [ ]:
""" df2["P_liza__c"] = df2["P_liza__c"].astype(str)
df2["P_liza__c"] = df2["P_liza__c"].str.replace(".", "")
df2["P_liza__c"] = df2["P_liza__c"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
df2["P_liza__c"] = df2["P_liza__c"].str.upper()
df2["P_liza__c"] = df2["P_liza__c"].str.strip() """

In [ ]:
modelo_nuevo = pd.merge(
    left=rutesi,
    right=df2,
    how="left",
    left_on="poliza",
    right_on="P_liza__c",
)
modelo_nuevo.to_excel("F:/Empresas SF/reportes/modelo_nuevo.xlsx", index=False)

In [ ]:
modelo = pd.DataFrame()
modelo = modelo_nuevo[["rutT", "Id Account Paciente", "Id"]]

In [ ]:
modelo.head()